In [1]:
import matplotlib.pyplot as plt

import keras
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub # can pip install

In [2]:
train_data_path = "train"
validation_data_path = "validation"

## Data Preprocessing

In [14]:
## data augmentation
training_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')

In [15]:
training_data =  training_datagen.flow_from_directory(train_data_path, 
                                      target_size=(224,224),
#                                     color_mode="grayscale",
                                      batch_size=5,
                                      class_mode='binary')

Found 1027 images belonging to 2 classes.


In [16]:
training_data.class_indices

{'horses': 0, 'humans': 1}

In [17]:
training_data.shape

AttributeError: 'DirectoryIterator' object has no attribute 'shape'

In [18]:
valid_datagen = ImageDataGenerator(rescale=1./255)

In [19]:
valid_data =  valid_datagen.flow_from_directory(validation_data_path, 
                                      target_size=(224,224), 
#                                     color_mode="grayscale",
                                      batch_size=5,
                                      class_mode='binary')

Found 256 images belonging to 2 classes.


## Show Imge

In [9]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

In [10]:
BATCH_SIZE = 5
IMG_SHAPE = 224 # match image dimension to mobile net input

### Using Transfer Learning Mobilenet

In [11]:
# getting MobileNet
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
mobile_net = hub.KerasLayer(URL, input_shape=(IMG_SHAPE, IMG_SHAPE, 1))

In [12]:
mobile_net.trainable = False

In [13]:
model = tf.keras.models.Sequential([
    mobile_net,
    tf.keras.layers.Dense(2, activation='softmax') #[0, 1] or [1, 0]
    ])

ValueError: in user code:

    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_hub\keras_layer.py:235 call  *
        result = smart_cond.smart_cond(training,
    C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\saved_model\load.py:668 _call_attribute  **
        return instance.__call__(*args, **kwargs)
    C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\eager\def_function.py:828 __call__
        result = self._call(*args, **kwds)
    C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\eager\def_function.py:871 _call
        self._initialize(args, kwds, add_initializers_to=initializers)
    C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\eager\def_function.py:726 _initialize
        *args, **kwds))
    C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\eager\function.py:2969 _get_concrete_function_internal_garbage_collected
        graph_f

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
EPOCHS = 1

In [ ]:
import numpy as np
history = model.fit_generator(
    training_data,
    epochs=EPOCHS,
    validation_data=valid_data
    )

In [ ]:
#save model
from tensorflow.keras.models import load_model
model.save('horsehuman.h5')

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

## Using CNN Model

In [ ]:
import numpy as np
from keras.preprocessing import image

In [ ]:
def pred_human_horse(model, horse_or_human):
  test_image = image.load_img(horse_or_human, target_size = (224,224))
  test_image = image.img_to_array(test_image)/255
  test_image = np.expand_dims(test_image, axis = 0)
 
  result = model.predict(test_image).round(3)
 
  pred = np.argmax(result)
  print(result, "--->>>", pred)
 
  if pred == 0:
    print('Predicted>>> Horse')
  else:
    print('Predicted>>> Human')

In [ ]:
pic = 'h.jfif'
pred_human_horse(model, pic)

### Building cnn model

In [20]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
##
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

In [21]:
#Building cnn model
cnn_model = keras.models.Sequential([
                                    keras.layers.Conv2D(filters=32, kernel_size=3, input_shape=[224,224, 3]),
                                    keras.layers.MaxPooling2D(pool_size=(2,2)),
    
                                    keras.layers.Conv2D(filters=64, kernel_size=3),
                                    keras.layers.MaxPooling2D(pool_size=(2,2)),
    
                                    keras.layers.Conv2D(filters=128, kernel_size=3),
                                    keras.layers.MaxPooling2D(pool_size=(2,2)), 
    
                                    keras.layers.Conv2D(filters=256, kernel_size=3),
                                    keras.layers.MaxPooling2D(pool_size=(2,2)),

                                    keras.layers.Dropout(0.5),                                                                        
                                    keras.layers.Flatten(), # neural network beulding
                                    keras.layers.Dense(units=128, activation='relu'), # input layers
                                    keras.layers.Dropout(0.1),                                    
                                    keras.layers.Dense(units=256, activation='relu'),                                    
                                    keras.layers.Dropout(0.25),                                    
                                    keras.layers.Dense(units=2, activation='softmax') # output layer
])

In [22]:
# compile cnn model
cnn_model.compile(optimizer = Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
model_path = 'horse_human_rgb.h5'
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [24]:
# train cnn model
history = cnn_model.fit(training_data, 
                          epochs=1, 
                          verbose=1, 
                          validation_data= valid_data,
                          callbacks=callbacks_list)

206/206 [==============================] - 138s 662ms/step - loss: 1.1402 - accuracy: 0.5658 - val_loss: 0.7180 - val_accuracy: 0.5391

Epoch 00001: val_accuracy improved from -inf to 0.53906, saving model to horse_human_rgb.h5


In [26]:
model_json = cnn_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model_path1 = 'horse_or_human_predictor.h5' #new model.ipynb

In [ ]:
model1 = keras.models.load_model(model_path1)

In [ ]:
import numpy as np
from keras.preprocessing import image

In [ ]:
def pred_human_horse(model, horse_or_human):
  test_image = image.load_img(horse_or_human, target_size = (224,224))
  test_image = image.img_to_array(test_image)/255
  test_image = np.expand_dims(test_image, axis = 0)
 
  result = cnn_model.predict(test_image).round(3)
 
  pred = np.argmax(result)
  print(result, "--->>>", pred)
 
  if pred == 0:
    print('Predicted>>> Horse')
  else:
    print('Predicted>>> Human')

In [ ]:
# #horse image path
# h1 = 'validation/horses/horse1-105.png'
# #human image path
# hu1 = 'validation/humans/valhuman01-01.png'
# for horse_or_human in [h1,hu1]:
#   pred_human_horse(model1, horse_or_human)

In [ ]:
pic = 'h.jfif'
pred_human_horse(model, pic)